In [16]:
from database import *
import psycopg2
from HST_variations import *
from psycopg2.extras import register_json
import pandas as pd
from io import StringIO
import os
import pandas as pd
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [17]:
dbHost = '13.36.221.162'
dbName = 'thminsight'
dbUser = 'thminsight'
dbPort = '5432'
dbPassword = 'psS8hqr6oaoJ7fgk'

In [18]:
db=Database(dbHost, dbName, dbUser,dbPassword,dbPort )
queryManager = Query(dbHost, dbName, dbUser,dbPort, dbPassword)

Database Ready
Database Ready


In [19]:
id_temperature_var = 1245
id_water_level_var = 1246
id_tv_var=1238
id_new_var=1238
id_new_var2=1238
id_res_var = 0
res_metric_id = 47
res_name = 'res'

In [23]:
model=[]
dataframes=[]
def get_targetvr_df(id_tv_var):
    target_column= 'target_variable'
    target_variable_name=queryManager.get_variable_name_by_id(id_tv_var)
    df_def = queryManager.get_df_by_variable_id(id_tv_var, target_column)
    assert queryManager.get_variable_metric(id_temperature_var)=='°C'
    dataframes.append(df_def)
    return df_def
def get_temp_df(id_temperature_var):
    df_temp= queryManager.get_df_by_variable_id(id_temperature_var, 'temperature')
    model.append('temp')
    dataframes.append(df_temp)
    return df_temp,model
def get_waterlevel_df(id_tv_var):
    df_def = queryManager.get_df_by_variable_id(id_water_level_var, 'water_level')
    assert queryManager.get_variable_metric(id_water_level_var)=='mA'
    model.append('H')
    dataframes.append(df_def)
    return df_def,model
def get_var_df(id_vr):
    variable_name=queryManager.get_variable_name_by_id(id_tv_var)
    df_def = queryManager.get_df_by_variable_id(id_vr, variable_name)
    model.append(variable_name)
    dataframes.append(df_def)
    return df_def,model    

In [37]:
json_path='./test.json'
f = open(json_path)

In [24]:
get_targetvr_df(id_tv_var)
get_waterlevel_df(id_tv_var)
get_temp_df(id_temperature_var)
get_var_df(id_new_var)
#get_var_df(id_new_var2)

(                     timestamp  Distance
 0    2022-02-19 09:28:36+00:00      -1.0
 1    2022-02-18 05:08:44+00:00      -1.0
 2    2022-02-23 01:48:13+00:00      -1.0
 3    2022-02-22 08:28:17+00:00      -1.0
 4    2022-02-22 22:28:14+00:00      -1.0
 ...                        ...       ...
 5049 2022-04-12 12:04:07+00:00      -1.0
 5050 2022-04-12 12:44:06+00:00      -1.0
 5051 2022-04-12 13:04:06+00:00      -1.0
 5052 2022-04-12 13:24:06+00:00      -1.0
 5053 2022-04-12 14:04:06+00:00      -1.0
 
 [5054 rows x 2 columns],
 ['H', 'temp', 'Distance'])

In [29]:
model+=['T','S']
model

['H', 'temp', 'Distance', 'T', 'S']

In [30]:
df=final_prep(dataframes,option='tolerance sampling')
df.head()

,temperature,target_variable,water_level,Distance,Time
0,-3.059853,-1.0,5.380181,-1.0,2021-12-22 07:36:25
1,-2.744339,-1.0,5.380181,-1.0,2021-12-22 07:56:26
2,-2.375446,-1.0,5.379704,-1.0,2021-12-22 08:16:25
3,-1.943739,-1.0,5.375498,-1.0,2021-12-22 08:36:35
4,-1.393096,-1.0,5.376279,-1.0,2021-12-22 08:56:25


In [31]:
optimizers_list=['SGD','Adam', 'AdamW','Adamax','ASGD','Adagrad']
methods_list=['std_regression',
              'std_regression_cross',
              'use_NN_optimizer',
              'use_NN_optimizer_cross',
             'use_bayesian_approch']

In [32]:
for method,optimizer in zip(methods_list,optimizers_list):
    print(method,optimizer)
    res=deploy(model,df,method,optimizer)
res.head()

std_regression SGD
std_regression_cross Adam
use_NN_optimizer AdamW
use_NN_optimizer_cross Adamax
use_bayesian_approch ASGD


,corrected observation,temperature,target_variable,water_level,Distance,Time
0,-1.637105e-10,-3.059853,-1.0,5.380181,-1.0,2021-12-22 07:36:25
1,-1.649122e-10,-2.744339,-1.0,5.380181,-1.0,2021-12-22 07:56:26
2,-1.679672e-10,-2.375446,-1.0,5.379704,-1.0,2021-12-22 08:16:25
3,-1.692284e-10,-1.943739,-1.0,5.375498,-1.0,2021-12-22 08:36:35
4,-1.720514e-10,-1.393096,-1.0,5.376279,-1.0,2021-12-22 08:56:25


In [33]:
id_res_var=180
df_res =prepare_corrected_obs(res,id_res_var)

In [34]:
df_res.head()

,timestamp,value,variable_id
0,2021-12-22 07:36:25,-1.637105e-10,180
1,2021-12-22 07:56:26,-1.649122e-10,180
2,2021-12-22 08:16:25,-1.679672e-10,180
3,2021-12-22 08:36:35,-1.692284e-10,180
4,2021-12-22 08:56:25,-1.720514e-10,180


In [35]:
queryManager.deleteData(id_res_var, '1967-01-01 00:00:00.00+02:00')
queryManager.db.insertDataframe(df_res, 'raw_data_save') 

Insert dataframe done with success!
